In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import datetime
from datetime import timedelta
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
import os

In [3]:
# os.chdir('/media/minwookim/9C33-6BBD/WorkSpace/Next_Ez_jejudo/data/골프장데이터')리눅스

In [6]:
df = pd.read_csv('골프장예약만데이터ver0303.csv')
df.head(3)

,골프장명,골프장지역명,골프장멤버십값,골프장유형값,사용일자,사용시간값,사용유형명,거주자금액,비거주자금액,결제수단명,...,예약자골프경력값,예약자평균사용값,예약자사용성향값,요일별,계절,월,시간대,예약시간대,일,해당년도
0,해비치CC,동부권,회원제+대중제,36,20210120,815,SKY,170000,170000,현장결제,...,NaN,NaN,NaN,수요일,겨울,1월,아침,아침,20일,2021년
1,아덴힐GC,서부권,대중제,18,20210121,809,왕이메,90000,135000,현장결제,...,NaN,NaN,NaN,목요일,겨울,1월,아침,아침,21일,2021년
2,아덴힐GC,서부권,대중제,18,20210126,823,왕이메,90000,135000,현장결제,...,NaN,NaN,NaN,화요일,겨울,1월,아침,아침,26일,2021년


### 1. 부킹시간대
01 ~ 07 = [1,2,3,4,5,6,7]

08 ~ 12 = [8,9,10,11,12]

13 ~ 18 = [13,14,15,16,17,18]

19 ~ 24 = [19,20,21,22,23,24]

### 2. 사전예약(취소) 통계
당일예약

1~2일전

3~7일전

8~15일

16일~


### 3.사용일 통계

시간대/일자

06시~08시

09시 ~ 10시

11시~12시

13시 ~


### 4. 가격대 별 예약(취소) 통계
- 도민, 비도민 나누기
90,000 이하

90,000~,110000

110,000~120,000

120,000~130,000

130,000~140,000

140,000~160,000

160,000 ~ 180,000
180,000 ~ 200,000
200,000원 이상 


### 5. 전화 연결 통계

시간대/일자

~ 09시 

09시 ~ 12시

13시 ~ 18시

19시 ~ 24시


In [16]:
df_cancel = pd.read_csv('/media/minwookim/9C33-6BBD/WorkSpace/Next_Ez_jejudo/data/골프장데이터/제주골프장_예약취소_2112-2202.csv',encoding='euc-KR',header=None)

In [17]:
col_list = '''골프장명
골프장지역명
골프장멤버십값
골프장유형값
사용일자
사용시간값
사용유형명
거주자금액
비거주자금액
결제수단명
총인원수
거주자인원수
비거주자인원수
예약일자
예약시간
취소일자
취소시간
판매업체명
예약자성별코드
예약자연령대코드
예약자거주지값
예약자평균값
예약자골프경력값
예약자평균사용값
예약자사용성향값'''.split('\n')

In [18]:
len(col_list)

25

In [19]:
len(df_cancel.columns)

25

In [21]:
df_cancel.columns=col_list

In [22]:
df_cancel.총인원수.unique()

array([4])

In [23]:
df_cancel.columns

Index(['골프장명', '골프장지역명', '골프장멤버십값', '골프장유형값', '사용일자', '사용시간값', '사용유형명',
       '거주자금액', '비거주자금액', '결제수단명', '총인원수', '거주자인원수', '비거주자인원수', '예약일자', '예약시간',
       '취소일자', '취소시간', '판매업체명', '예약자성별코드', '예약자연령대코드', '예약자거주지값', '예약자평균값',
       '예약자골프경력값', '예약자평균사용값', '예약자사용성향값'],
      dtype='object')

## 취소 

In [ ]:
df_cancel.취소일자[0]-df_cancel.예약일자[0]

In [34]:
df_cancel.취소일자[0]

20211201

In [35]:
df_cancel.예약일자[0]

20211117

## 당일예약

## 참고 

In [ ]:
df['월'] = df['사용일자'].apply(lambda x: str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [3,4,5]) else 
                                    str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [6,7,8]) else 
                                   str(x)[4:-2].lstrip('0')+'월' if ( int(str(x)[4:-2].lstrip('0')) in [9,10,11]) else 
                                    str(x)[4:-2].lstrip('0')+'월'  if ( int(str(x)[4:-2].lstrip('0')) in [12,1,2]) else '미분류')

In [13]:
days=datetime.date(2021,12,1)-datetime.date(2021,11,17)
days.days == 14

True